In [ ]:
import pandas as pd
matches = pd.read_csv("matches.csv")

# changes date to not be an object so that it can be fed to model
matches["date"] = pd.to_datetime(matches["date"]) 

# converts home/away column into numeric column 
matches["venue_code"] = matches["venue"].astype("category").cat.codes

# numeric indicator for each team 
matches["opp_code"] = matches["opponent"].astype("category").cat.codes

# numeric indicator for when a team plays their game 
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")

# number for each day of the week
matches["day_code"] = matches["date"].dt.dayofweek

# predict whether the team won or not 
matches["target"] = (matches["result"] == 'W').astype("int")

# training model
from sklearn.ensemble import RandomForestClassifier

# n number of decision trees we want to train 
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

# we split train and test so that model has new data to see if its correct or not
train = matches[matches["date"] < '2024-01-01']
test = matches[matches["date"] > '2024-01-01']
predictors = ["venue_code", "opp_code", "hour", "day_code"]

rf.fit(train[predictors], train["target"])
RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

predictions = rf.predict(test[predictors])

# determine accuracy of the model
from sklearn.metrics import accuracy_score
acur = accuracy_score(test["target"], predictions)

# to see where we were accurate and where we were not
combined = pd.DataFrame(dict(actual=test["target"], prediction=predictions))

# pd.crosstab(index=combined["actual"], columns=combined["prediction"])   
from sklearn.metrics import precision_score
precision_score(test["target"], predictions)

# creates more predictors to improve accuracy (rolling averages)
# uses a team's form to predict as well
grouped_matches = matches.groupby("team")
group = grouped_matches.get_group("Manchester City")

def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed='left').mean() # closed takes current week out
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols]

# takes dataframes and groups 1 dataframe for each team and compute rolling averages using func
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))
matches_rolling = matches_rolling.droplevel('team')
matches_rolling.index = range(matches_rolling.shape[0])

# new set of predictors
def make_predictions(data, predictors):
    train = data[data["date"] < '2024-01-01']
    test = data[data["date"] > '2024-01-01']
    rf.fit(train[predictors], train["target"])
    predictions = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test['target'], predicted=predictions), index=test.index)
    precision = precision_score(test["target"], predictions)
    return combined, precision

combined, precision = make_predictions(matches_rolling, predictors + new_cols)

combined = combined.merge(
    matches_rolling[["date", "team", "opponent", "result"]],
    left_index=True,
    right_index=True
)

# There may be cases for a match where the predictor predicted both teams would win or lose the same match 
# we need to combine those games

# class that inherits from the dict class
class MissingDict(dict):
    __missing__ = lambda self, key: key

map_values = {
    "Brighton and Hove Albion": "Brighton",
    "Manchester United": "Manchester Utd", 
    "Newcastle United": "Newcastle Utd", 
    "Tottenham Hotspur": "Tottenham",
    "West Ham United": "West Ham",
    "Wolverhampton Wanderers": "Wolves"
}

mapping = MissingDict(**map_values)

combined["new_team"] = combined["team"].map(mapping)

merged = combined.merge(
    combined,
    left_on=["date", "new_team"],
    right_on=["date", "opponent"]
)

merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] == 0)]["actual_x"].value_counts()




/var/folders/fq/_dff320n2s97p0x95sv8srdr0000gn/T/ipykernel_17156/1831718174.py:65: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))


0.5724907063197026